In [1]:
import os
import random
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator



ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Tentukan path ke direktori data
data_dir = 'private/tomatect/'
classes = ['bacterial_spot', 'early_blight', 'healthy', 'leaf_mold', 'spider_mites']

In [ ]:
# Tentukan parameter untuk pembagian data
train_ratio = 0.8
val_ratio = 0.15
test_ratio = 0.05

In [ ]:

# Inisialisasi list untuk menyimpan data train, validation, dan test
train_images = []
train_labels = []
val_images = []
val_labels = []
test_images = []
test_labels = []

In [ ]:
# Loop melalui setiap kelas
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    images = os.listdir(class_dir)
    random.shuffle(images)
    
    # Hitung jumlah data pada setiap split
    num_images = len(images)
    num_train = int(num_images * train_ratio)
    num_val = int(num_images * val_ratio)
    num_test = num_images - num_train - num_val
    
    # Bagi data menjadi train, validation, dan test
    train_images.extend(images[:num_train])
    train_labels.extend([class_name] * num_train)
    val_images.extend(images[num_train:num_train+num_val])
    val_labels.extend([class_name] * num_val)
    test_images.extend(images[num_train+num_val:])
    test_labels.extend([class_name] * num_test)

In [ ]:
# Inisialisasi objek ImageDataGenerator dengan augmentasi yang diinginkan
datagen = ImageDataGenerator(
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    rotation_range=90
)

In [ ]:
# Fungsi untuk memuat dan augmentasi citra
def load_and_augment_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.resize(image, (256, 256))  # Resize citra sesuai dengan kebutuhan Anda
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konversi dari BGR ke RGB
    image = datagen.random_transform(image)
    return image

In [ ]:
# Fungsi generator untuk menghasilkan citra secara batch
def image_generator(image_paths, labels, batch_size=32):
    num_samples = len(image_paths)
    while True:
        indices = np.random.permutation(num_samples)
        batch_start = 0
        while batch_start < num_samples:
            batch_indices = indices[batch_start:batch_start+batch_size]
            batch_images = [load_and_augment_image(image_paths[i]) for i in batch_indices]
            batch_labels = [labels[i] for i in batch_indices]
            yield np.array(batch_images), np.array(batch_labels)
            batch_start += batch_size


In [ ]:
# Inisialisasi model CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 32)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(classes), activation='softmax'))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-06 23:11:56.733258: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-06 23:11:56.733597: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Buat generator untuk data train, validation, dan test
train_generator = image_generator(train_images, train_labels, batch_size=32)
val_generator = image_generator(val_images, val_labels, batch_size=32)
test_generator = image_generator(test_images, test_labels, batch_size=32)

In [ ]:
# Hitung jumlah batch per epoch
steps_per_epoch = len(train_images) // 32
validation_steps = len(val_images) // 32


In [ ]:
# Pelatihan model
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10,
          validation_data=val_generator, validation_steps=validation_steps)

FileNotFoundError: [Errno 2] No such file or directory: 'leaf_mold_30.jpg'